### Setup

In this section, we load necessary libraries and define custom functions.

In [1]:
# install PPI library if needed 
# %pip install git+https://github.com/Michael-Howes/ppi_py.git
import pandas as pd
import numpy as np
import random
import statsmodels.api as sm
import sys
from scipy import stats
from ppi_py import ppi_ols_ci, classical_ols_ci, ppi_ols_pointestimate
import PythonFunctions as pf


/var/folders/md/0h30crhs7xb714g8zy7dzj1m0000gn/T/ipykernel_18552/214817625.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv("../Data/4_gpt4turbo_wp_20241118.csv.gz", dtype={'UserID': 'string'})

Covs = ['PedPed', 'Barrier', 'CrossingSignal', 'NumberOfCharacters',
        'DiffNumberOFCharacters', 'LeftHand', 'Man', 'Woman', 'Pregnant',
        'Stroller', 'OldMan', 'OldWoman', 'Boy', 'Girl', 'Homeless',
        'LargeWoman', 'LargeMan', 'Criminal', 'MaleExecutive',
        'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete', 'FemaleDoctor',
        'MaleDoctor', 'Dog', 'Cat', 
        'Intervention'
        ]

sys.version

'3.11.4 (v3.11.4:d2340ef257, Jun  6 2023, 19:15:51) [Clang 13.0.0 (clang-1300.0.29.30)]'

In [3]:
print("Number of respondents: ", len(df["UserID"].unique()))
print("Number of scenarios: ", len(df["ResponseID"].unique()))
print("Number of options per scenario: ", df["ResponseID"].value_counts().unique()[0])
print("Avg. number of scenarios per respondent: ", round(df.groupby("UserID")["ResponseID"].count().mean()/2,0))
print("Number of rows:", df.shape[0])
print("Number of NAs in observed dependent variable: ", df["Saved"].isna().sum())
print("Number of NAs in predicted dependent variable with GPT4 Turbo: ", df["gpt4turbo_wp_Saved"].isna().sum())

Number of respondents:  54695
Number of scenarios:  581981
Number of options per scenario:  2
Avg. number of scenarios per respondent:  11.0
Number of rows: 1163962
Number of NAs in observed dependent variable:  0
Number of NAs in predicted dependent variable with GPT4 Turbo:  0


### Reproduce AMCE from R functions and define 'ground truth' for simulation

Below we compute the Average Marginal Component Effect (AMCE) for an attribute of the moral dilemmas using  weighted least squares. Awad et al. (2018) use R to estimate the AMCE for the the conjoint experiment. First, we verify that we can obtain the results with our Python code. 

In [4]:
amce_human_subjects = pd.concat([
    pf.compute_amce(df, x="Intervention", y="Saved"), 
    pf.compute_amce(df, x="Barrier", y="Saved"), 
    pf.compute_amce(df, x="Gender", y="Saved"), 
    pf.compute_amce(df, x="Fitness", y="Saved"), 
    pf.compute_amce(df, x="Social Status", y="Saved"), 
    pf.compute_amce(df, x="CrossingSignal",y="Saved"),
    pf.compute_amce(df, x="Age", y="Saved"),
    pf.compute_amce(df, x="Utilitarian", y="Saved"),
    pf.compute_amce(df, x="Species", y="Saved")
])      
amce_human_subjects.round(3)

,x,y,beta,se,lower,upper
0,Intervention,Saved,0.081,0.002,0.078,0.084
0,Barrier,Saved,0.105,0.003,0.100,0.111
0,Gender,Saved,0.135,0.003,0.129,0.142
0,Fitness,Saved,0.176,0.004,0.169,0.183
0,Social Status,Saved,0.240,0.009,0.222,0.258
0,CrossingSignal,Saved,0.378,0.003,0.372,0.383
0,Age,Saved,0.508,0.003,0.501,0.514
0,Utilitarian,Saved,0.571,0.003,0.565,0.576
0,Species,Saved,0.684,0.003,0.679,0.690


The AMCE estimates above are the same as those calculated with the R functions by Awad et al. (2018), see object `main.Saved` and `7_AmceParamsSimulationR.csv.tar` created in the R script `7_CalculateAMCE.R`. Hence, the Python functions defined in this notebook give the same results as the functions defined in the original article. 

We use these AMCEs computed on the human subjects as the 'ground truth' in our simulation.

### Estimate the PPI correlation

Below we estimate the PPI correlation with a large number of human subjects. 

In [5]:
n = 100000
N = df.shape[0] - n

# Sample decisions for human subjects sample
df_human = df.sample(n=n, replace = False, random_state=2024)

# Drop these decisions from the full dataset
df_remaining = df.drop(df_human.index)

# Sample decisions for silicon sample
df_silicon = df_remaining.sample(n=N, replace = False, random_state=2024)

model = "gpt4turbo_wp_Saved"

results = pd.concat([
    pf.compute_amce_ppi(df_human, df_silicon, x="Intervention", y=model), 
    pf.compute_amce_ppi(df_human, df_silicon, x="Barrier", y=model), 
    pf.compute_amce_ppi(df_human, df_silicon, x="Gender", y=model), 
    pf.compute_amce_ppi(df_human, df_silicon, x="Fitness", y=model), 
    pf.compute_amce_ppi(df_human, df_silicon, x="Social Status", y=model), 
    pf.compute_amce_ppi(df_human, df_silicon, x="CrossingSignal",y=model),
    pf.compute_amce_ppi(df_human, df_silicon, x="Age", y=model),
    pf.compute_amce_ppi(df_human, df_silicon, x="Utilitarian", y=model),
    pf.compute_amce_ppi(df_human, df_silicon, x="Species", y=model)
],ignore_index=True)
    
results.to_csv("../Data/7_rho.csv", index=False)
results

,y,x,beta_ppi,beta_hum,beta_sil,beta_sil_sm,se_ppi,se_hum,se_sil,lower_ppi,upper_ppi,lower_hum,upper_hum,lower_sil,upper_sil,ppi_corr
0,gpt4turbo_wp_Saved,Intervention,0.078747,0.078896,0.085381,0.085381,0.003496,0.003714,0.001105,0.071895,0.085599,0.071617,0.086176,0.083215,0.087546,0.355201
1,gpt4turbo_wp_Saved,Barrier,0.106857,0.105027,0.487042,0.487042,0.005341,0.005609,0.001443,0.096390,0.117325,0.094034,0.116021,0.484213,0.489870,0.313918
2,gpt4turbo_wp_Saved,Gender,0.138434,0.139610,0.201058,0.201058,0.008338,0.008576,0.002512,0.122088,0.154770,0.122802,0.156419,0.196134,0.205982,0.244746
3,gpt4turbo_wp_Saved,Fitness,0.191511,0.190903,0.022484,0.022484,0.008669,0.008988,0.002661,0.174518,0.208501,0.173287,0.208519,0.017268,0.027701,0.274928
4,gpt4turbo_wp_Saved,Social Status,0.251152,0.248660,0.042553,0.042553,0.023434,0.023867,0.007165,0.205176,0.297037,0.201881,0.295440,0.028510,0.056595,0.200823
5,gpt4turbo_wp_Saved,CrossingSignal,0.373724,0.374077,0.656967,0.656967,0.005964,0.006242,0.001563,0.362035,0.385414,0.361843,0.386311,0.653905,0.660030,0.308616
6,gpt4turbo_wp_Saved,Age,0.515574,0.518050,0.179852,0.179852,0.007526,0.007675,0.002570,0.500825,0.530325,0.503008,0.533092,0.174815,0.184889,0.207828
7,gpt4turbo_wp_Saved,Utilitarian,0.578739,0.579395,0.554620,0.554620,0.006957,0.007092,0.002154,0.565098,0.592369,0.565495,0.593295,0.550398,0.558843,0.203924
8,gpt4turbo_wp_Saved,Species,0.692029,0.691949,0.844766,0.844766,0.006249,0.006252,0.001486,0.679783,0.704279,0.679696,0.704202,0.841854,0.847678,0.044067


### Simulation

Next, we vary the number of human subjects and silicon subjects in a simulation.

In [6]:
if __name__ == "__main__":
    
    from multiprocessing import Pool

    # Initialize multiprocessing Pool
    num_cores = 9

    # structural attributes of scenarios
    Xs_structural  = ['Intervention', 'Barrier','CrossingSignal']

    # attributes of characters
    Xs_characters = ['Gender','Fitness','Social Status','Age','Utilitarian','Species']

    # all attributes
    Xs = Xs_structural + Xs_characters

    with Pool(processes=num_cores) as pool:

        # Map the process_predictor function to all predictors
        results = pool.map(pf.loop_attribute, Xs)

    # Combine all results into a single DataFrame
    result = pd.concat(results, ignore_index=True)

Scenario attribute: Utilitarian
Scenario attribute: Fitness
Scenario attribute: Social Status
Scenario attribute: Gender
Scenario attribute: Barrier
Scenario attribute: Intervention
Scenario attribute: Age
Scenario attribute: CrossingSignal
Scenario attribute: Species
    Silicon sample size: 1000
    Silicon sample size: 1000
    Silicon sample size: 1000
    Silicon sample size: 1000
    Silicon sample size: 1000
    Silicon sample size: 1000
    Silicon sample size: 1000
    Silicon sample size: 1000
    Silicon sample size: 1000
    Silicon sample size: 2500
    Silicon sample size: 2500
    Silicon sample size: 2500
    Silicon sample size: 2500
    Silicon sample size: 2500
    Silicon sample size: 2500
    Silicon sample size: 2500
    Silicon sample size: 2500
    Silicon sample size: 2500
    Silicon sample size: 5000
    Silicon sample size: 5000
    Silicon sample size: 5000
    Silicon sample size: 5000
    Silicon sample size: 7500
    Silicon sample size: 5000
    Silicon

In [ ]:
result

,y,x,beta_ppi,beta_hum,beta_sil,beta_sil_sm,se_ppi,se_hum,se_sil,lower_ppi,upper_ppi,lower_hum,upper_hum,lower_sil,upper_sil,ppi_corr,n,N,reps
0,gpt4turbo_wp_Saved,Intervention,0.095979,0.096036,0.078320,0.078320,0.011616,0.011684,0.035492,0.073212,0.118745,0.073136,0.118935,0.008757,0.147884,0.341377,10000,1000,500
1,gpt4turbo_wp_Saved,Intervention,0.067066,0.066761,0.099217,0.099217,0.011651,0.011723,0.036240,0.044229,0.089899,0.043785,0.089737,0.028189,0.170246,0.368784,10000,1000,500
2,gpt4turbo_wp_Saved,Intervention,0.076389,0.080568,-0.048944,-0.048944,0.011688,0.011751,0.036356,0.053474,0.099290,0.057537,0.103600,-0.120200,0.022312,0.348172,10000,1000,500
3,gpt4turbo_wp_Saved,Intervention,0.085028,0.084467,0.092921,0.092921,0.011633,0.011707,0.034987,0.062233,0.107834,0.061521,0.107413,0.024346,0.161495,0.354470,10000,1000,500
4,gpt4turbo_wp_Saved,Intervention,0.074795,0.076444,0.039091,0.039091,0.011645,0.011704,0.035560,0.051956,0.097603,0.053504,0.099383,-0.030606,0.108788,0.336188,10000,1000,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103495,gpt4turbo_wp_Saved,Species,0.686263,0.686263,0.849634,0.849634,0.020124,0.020118,0.004779,0.646821,0.725705,0.646832,0.725694,0.840268,0.859001,0.055698,10000,100000,500
103496,gpt4turbo_wp_Saved,Species,0.673780,0.673780,0.848726,0.848726,0.021148,0.021142,0.004732,0.632331,0.715228,0.632343,0.715217,0.839451,0.858001,0.013369,10000,100000,500
103497,gpt4turbo_wp_Saved,Species,0.681409,0.680958,0.849563,0.849563,0.019950,0.019944,0.004752,0.641857,0.720059,0.641868,0.720048,0.840248,0.858877,0.003471,10000,100000,500
103498,gpt4turbo_wp_Saved,Species,0.691178,0.691544,0.848563,0.848563,0.019715,0.019914,0.004859,0.652538,0.729818,0.652513,0.730574,0.839039,0.858087,0.148275,10000,100000,500


We benchmark the silicon subjects design and the mixed subjects design against a human subjects approach.

In [ ]:
# subset point estimates of AMCEs from the entire human subjects sample
benchmark = amce_human_subjects.loc[:, ['x', 'beta']].rename(columns={'beta': 'param'})

# merge benchmark with results from simulation
result_wb = pd.merge(result, benchmark, on='x', how='left')

# report if true value is within the confidence interval from the mixed subjects 
result_wb['coverage_ppi'] = (
    (result_wb['lower_ppi'] <= result_wb['param']) & 
    (result_wb['param'] <= result_wb['upper_ppi'])
).astype(int) 

# report if true value is within the confidence interval from the silicon subjects 
result_wb['coverage_sil'] = (
    (result_wb['lower_sil'] <= result_wb['param']) & 
    (result_wb['param'] <= result_wb['upper_sil'])
).astype(int) 

# report if true value is within the confidence interval from the silicon subjects 
result_wb['coverage_hum'] = (
    (result_wb['lower_hum'] <= result_wb['param']) & 
    (result_wb['param'] <= result_wb['upper_hum'])
).astype(int) 

result_wb

,y,x,beta_ppi,beta_hum,beta_sil,beta_sil_sm,se_ppi,se_hum,se_sil,lower_ppi,...,lower_sil,upper_sil,ppi_corr,n,N,reps,param,coverage_ppi,coverage_sil,coverage_hum
0,gpt4turbo_wp_Saved,Intervention,0.095979,0.096036,0.078320,0.078320,0.011616,0.011684,0.035492,0.073212,...,0.008757,0.147884,0.341377,10000,1000,500,0.081441,1,1,1
1,gpt4turbo_wp_Saved,Intervention,0.067066,0.066761,0.099217,0.099217,0.011651,0.011723,0.036240,0.044229,...,0.028189,0.170246,0.368784,10000,1000,500,0.081441,1,1,1
2,gpt4turbo_wp_Saved,Intervention,0.076389,0.080568,-0.048944,-0.048944,0.011688,0.011751,0.036356,0.053474,...,-0.120200,0.022312,0.348172,10000,1000,500,0.081441,1,0,1
3,gpt4turbo_wp_Saved,Intervention,0.085028,0.084467,0.092921,0.092921,0.011633,0.011707,0.034987,0.062233,...,0.024346,0.161495,0.354470,10000,1000,500,0.081441,1,1,1
4,gpt4turbo_wp_Saved,Intervention,0.074795,0.076444,0.039091,0.039091,0.011645,0.011704,0.035560,0.051956,...,-0.030606,0.108788,0.336188,10000,1000,500,0.081441,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103495,gpt4turbo_wp_Saved,Species,0.686263,0.686263,0.849634,0.849634,0.020124,0.020118,0.004779,0.646821,...,0.840268,0.859001,0.055698,10000,100000,500,0.684374,1,0,1
103496,gpt4turbo_wp_Saved,Species,0.673780,0.673780,0.848726,0.848726,0.021148,0.021142,0.004732,0.632331,...,0.839451,0.858001,0.013369,10000,100000,500,0.684374,1,0,1
103497,gpt4turbo_wp_Saved,Species,0.681409,0.680958,0.849563,0.849563,0.019950,0.019944,0.004752,0.641857,...,0.840248,0.858877,0.003471,10000,100000,500,0.684374,1,0,1
103498,gpt4turbo_wp_Saved,Species,0.691178,0.691544,0.848563,0.848563,0.019715,0.019914,0.004859,0.652538,...,0.839039,0.858087,0.148275,10000,100000,500,0.684374,1,0,1


In [ ]:
# Group by n, N, and LLM then calculate mean across repetitions
vars = ['beta_ppi','se_ppi','lower_ppi','upper_ppi','coverage_ppi','ppi_corr',
        'beta_sil','se_sil','lower_sil','upper_sil','coverage_sil',
        'beta_hum','se_hum','lower_hum','upper_hum','coverage_hum']

summ = result_wb.groupby(['n','N','y','x','param'])[vars].mean().reset_index()

# Calculate bias columns
summ['bias_ppi'] = summ['beta_ppi'] - summ['param']
summ['bias_sil'] = summ['beta_sil'] - summ['param']
summ['bias_hum'] = summ['beta_hum'] - summ['param']

summ['rmse_ppi'] = np.sqrt(summ['bias_ppi']**2 + summ['se_ppi']**2)
summ['rmse_sil'] = np.sqrt(summ['bias_sil']**2 + summ['se_sil']**2)
summ['rmse_hum'] = np.sqrt(summ['bias_hum']**2 + summ['se_hum']**2)

# Save averaged simulation results to compressed csv file
summ.to_csv("../Data/7_ResultsPPI_coord1.csv.gz", compression="gzip", index=False)
summ

,n,N,y,x,param,beta_ppi,se_ppi,lower_ppi,upper_ppi,coverage_ppi,...,se_hum,lower_hum,upper_hum,coverage_hum,bias_ppi,bias_sil,bias_hum,rmse_ppi,rmse_sil,rmse_hum
0,10000,1000,gpt4turbo_wp_Saved,Age,0.507580,0.507178,0.024366,0.459300,0.554811,0.954,...,0.024406,0.459242,0.554912,0.954,-0.000402,-0.329111,-0.000503,0.024369,0.339371,0.024411
1,10000,1000,gpt4turbo_wp_Saved,Barrier,0.105468,0.106097,0.017606,0.071606,0.140622,0.942,...,0.017684,0.071301,0.140622,0.942,0.000629,0.384346,0.000493,0.017618,0.387192,0.017691
2,10000,1000,gpt4turbo_wp_Saved,CrossingSignal,0.377522,0.377854,0.019572,0.339488,0.416208,0.944,...,0.019654,0.339266,0.416308,0.940,0.000331,0.281382,0.000264,0.019575,0.285921,0.019656
3,10000,1000,gpt4turbo_wp_Saved,Fitness,0.175917,0.177808,0.028197,0.122471,0.233001,0.952,...,0.028292,0.122287,0.233191,0.952,0.001890,-0.153256,0.001822,0.028260,0.175878,0.028351
4,10000,1000,gpt4turbo_wp_Saved,Gender,0.135175,0.134544,0.026955,0.081618,0.187279,0.948,...,0.027035,0.081258,0.187234,0.952,-0.000632,0.071943,-0.000929,0.026962,0.108116,0.027051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,10000,100000,gpt4turbo_wp_Saved,Gender,0.135175,0.135353,0.026186,0.083977,0.186623,0.936,...,0.027045,0.082192,0.188206,0.932,0.000178,0.066644,0.000024,0.026186,0.067146,0.027045
203,10000,100000,gpt4turbo_wp_Saved,Intervention,0.081441,0.081154,0.011053,0.059488,0.102814,0.948,...,0.011731,0.058177,0.104160,0.942,-0.000287,0.003896,-0.000272,0.011057,0.005308,0.011734
204,10000,100000,gpt4turbo_wp_Saved,Social Status,0.240005,0.240834,0.073566,0.095995,0.384367,0.938,...,0.075673,0.091205,0.387839,0.926,0.000829,-0.198466,-0.000483,0.073570,0.199832,0.075675
205,10000,100000,gpt4turbo_wp_Saved,Species,0.684374,0.685640,0.020063,0.646292,0.724938,0.944,...,0.020100,0.646075,0.724866,0.944,0.001266,0.160336,0.001097,0.020103,0.160409,0.020130
